In [0]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Deconv2D
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 300
l = 10
num_filter = 20

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 272s 2us/step


In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [0]:
input = Input(shape=(img_height, img_width, channel,))

# Block 1
b1 = BatchNormalization()(input)
b1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', activation='relu', use_bias=False)(b1)
b1 = BatchNormalization()(b1)
b1 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', activation='relu', use_bias=False)(b1)
b1 = BatchNormalization()(b1)
b1 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', activation='relu', use_bias=False)(b1)
b1 = BatchNormalization()(b1)
b1 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_4', activation='relu', use_bias=False)(b1)
b1 = BatchNormalization()(b1)
b1 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_5', activation='relu', use_bias=False)(b1)
b1 = BatchNormalization()(b1)
b1 = MaxPooling2D(pool_size=(2,2))(b1)

# Block 2
b2 = BatchNormalization()(b1)
b2 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_6', activation='relu', use_bias=False)(b1)
b2 = BatchNormalization()(b2)
b2 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_7', activation='relu', use_bias=False)(b2)
b2 = BatchNormalization()(b2)
b2 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_8', activation='relu', use_bias=False)(b2)
b2 = BatchNormalization()(b2)
b2 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_9', activation='relu', use_bias=False)(b2)
b2 = BatchNormalization()(b2)
b2 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_10', activation='relu', use_bias=False)(b2)
b2 = BatchNormalization()(b2)
b2 = MaxPooling2D(pool_size=(2,2))(b2)

# Block 3
b3 = BatchNormalization()(b2)
b3 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_11', activation='relu', use_bias=False)(b3)
b3 = BatchNormalization()(b3)
b3 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_12', activation='relu', use_bias=False)(b3)
b3 = BatchNormalization()(b3)
b3 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_13', activation='relu', use_bias=False)(b3)
b3 = BatchNormalization()(b3)
b3 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_14', activation='relu', use_bias=False)(b3)
b3 = BatchNormalization()(b3)
b3 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_15', activation='relu', use_bias=False)(b3)
b3 = BatchNormalization()(b3)
b3 = MaxPooling2D(pool_size=(2,2))(b3)

# Block 4
b4 = BatchNormalization()(b3)
b4 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_16', activation='relu', use_bias=False)(b4)
b4 = BatchNormalization()(b4)
b4 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_17', activation='relu', use_bias=False)(b4)
b4 = BatchNormalization()(b4)
b4 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_18', activation='relu', use_bias=False)(b4)
b4 = BatchNormalization()(b4)
b4 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_19', activation='relu', use_bias=False)(b4)
b4 = BatchNormalization()(b4)
b4 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_20', activation='relu', use_bias=False)(b4)
b4 = BatchNormalization()(b4)
b4 = MaxPooling2D(pool_size=(2,2))(b4)

# Concatenate
x = BatchNormalization()(b4)
x = Deconv2D(512, 3, 3, output_shape=[None, 8, 8, 512], activation='relu', border_mode='same', subsample=(4,4))(x)
y = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)(b1)
y = LeakyReLU(alpha=0.1)(y)
y = Lambda(space_to_depth_x2)(y)
z = concatenate([x, y])
z = Flatten()(z)
output = Dense(num_classes, activation='softmax')(z)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: Update your `Conv2DTranspose` call to the Keras 2 API: `Conv2DTranspose(512, (3, 3), activation="relu", strides=(4, 4), padding="same")`


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_7 (BatchNor (None, 32, 32, 3)    12          input_6[0][0]                    
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 32, 32, 32)   864         batch_normalization_7[0][0]      
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 32, 32, 32)   128         conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Co

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [16]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/300
50000/50000 [==============================] - 255s 5ms/step - loss: 1.9845 - acc: 0.4613 - val_loss: 3.9598 - val_acc: 0.4560
Epoch 2/300
50000/50000 [==============================] - 237s 5ms/step - loss: 1.2945 - acc: 0.5848 - val_loss: 1.1879 - val_acc: 0.5913
Epoch 3/300
50000/50000 [==============================] - 236s 5ms/step - loss: 0.9798 - acc: 0.6687 - val_loss: 1.1108 - val_acc: 0.6334
Epoch 4/300
50000/50000 [==============================] - 237s 5ms/step - loss: 0.8463 - acc: 0.7165 - val_loss: 1.1348 - val_acc: 0.6340
Epoch 5/300
50000/50000 [==============================] - 237s 5ms/step - loss: 0.7194 - acc: 0.7591 - val_loss: 1.0966 - val_acc: 0.6664
Epoch 6/300
50000/50000 [==============================] - 237s 5ms/step - loss: 0.5914 - acc: 0.8020 - val_loss: 1.2828 - val_acc: 0.6517
Epoch 7/300
50000/50000 [==============================] - 237s 5ms/

KeyboardInterrupt: ignored

KeyboardInterrupt: ignored

In [17]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save the trained weights in to .h5 format
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")

10000/10000 [==============================] - 18s 2ms/step
Test loss: 3.22214862241745
Test accuracy: 0.6987
Saved the model to disk


In [19]:
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")
from google.colab import files

files.download('Yolo_Basic_model2.h5')

Saved the model to disk
